In [1]:
import copy
import numpy as np
import os
import json
import cv2
import sklearn.metrics as metric
from tensorflow.keras.models import Model
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def euclidean(a, b):
	return np.linalg.norm(a - b)

In [4]:
def perform_search(query_features, indexed_train, max_results=5):
	retrieved = []
	for idx in range(0, len(indexed_train["features"])):
		distance = euclidean(query_features, indexed_train["features"][idx])
		retrieved.append((distance, idx))
	retrieved = sorted(retrieved)[:max_results]
	return retrieved

In [5]:
base_dataset = "/content/drive/MyDrive/CBIR/TKC"
class_dir = ['Afrika', 'Bunga', 'Bus',
             'Dinosaurus', 'Gajah', 'Gedung',
             'Gunung', 'Kuda', 'Makanan','Pantai'
             ]

IMAGE_SIZE = (224, 224)

In [6]:
# indexing file images
dataset_test = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'test', class_item)
    for file in os.listdir(cur_dir):
        dataset_test.append(os.path.join(cur_dir, file))

In [7]:
print("len to retrieving:", len(dataset_test))

len to retrieving: 190


In [8]:
print("[INFO] load imageswang dataset...")
#  load images
test_images = []
for image_path in dataset_test:
    if ".jpg" or ".jpeg" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        test_images.append(image)

[INFO] load imageswang dataset...


In [9]:
print("[INFO] normalization...")
test_x = np.array(test_images).astype("float32") / 255.0

[INFO] normalization...


In [10]:
from tensorflow.keras.applications import VGG19
modelVGG19 = VGG19(weights="imagenet")
model = Model(inputs=modelVGG19.input, outputs=modelVGG19.get_layer("fc1").output)
model.summary()

574726144/574710816 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [11]:
features = model.predict(test_x)

In [12]:
with open('/content/drive/MyDrive/CBIR/TKC/feature_extraction2.json') as f:
  training_indexed = json.load(f)

In [13]:
query_indexes = list(range(0, test_x.shape[0]))
label_builder = list(np.unique(training_indexed["labels"]))
class_builder = {label_unique:[] for label_unique in label_builder}
recalls = copy.deepcopy(class_builder)
precisions = copy.deepcopy(class_builder)
# loop over the testing indexes
for i in query_indexes:
    queryFeatures = features[i]
    results = perform_search(queryFeatures, training_indexed, max_results=5)
    labels_ret = [training_indexed["labels"][r[1]] for r in results]
    label_true = dataset_test[i].split("/")[7]
    label_trues = [label_true for _ in labels_ret]
    recall = metric.recall_score(label_trues, labels_ret, average='weighted')
    precision = metric.precision_score(label_trues, labels_ret, average='weighted')
    recalls[label_true].append(recall)
    precisions[label_true].append(precision)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and 

In [14]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))

recall values:
Afrika 0.6315789473684209
Bunga 0.9894736842105264
Bus 1.0
Dinosaurus 1.0
Gajah 0.8
Gedung 0.6736842105263158
Gunung 0.7578947368421052
Kuda 0.8631578947368422
Makanan 0.7052631578947368
Pantai 0.4526315789473685
combined recall 0.7873684210526316

precision values:
Afrika 0.8947368421052632
Bunga 1.0
Bus 1.0
Dinosaurus 1.0
Gajah 1.0
Gedung 0.8421052631578947
Gunung 1.0
Kuda 1.0
Makanan 0.9473684210526315
Pantai 1.0
combined precision 0.968421052631579


In [15]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))

recall values:
Afrika 0.6315789473684209
Bunga 0.9894736842105264
Bus 1.0
Dinosaurus 1.0
Gajah 0.8
Gedung 0.6736842105263158
Gunung 0.7578947368421052
Kuda 0.8631578947368422
Makanan 0.7052631578947368
Pantai 0.4526315789473685
combined recall 0.7873684210526316

precision values:
Afrika 0.8947368421052632
Bunga 1.0
Bus 1.0
Dinosaurus 1.0
Gajah 1.0
Gedung 0.8421052631578947
Gunung 1.0
Kuda 1.0
Makanan 0.9473684210526315
Pantai 1.0
combined precision 0.968421052631579
